필요한 라이브러리 import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install autogluon -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from autogluon.text import TextPredictor

데이터 load

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/잡코리아_자소서_최종.csv')[['A']]
test.head()

,A
0,꾸준함꾸준함은 저 자신을 한 단어로 정의한다고 할 때 가장 먼저 표현할 수 있는 단...
1,어렸을 때부터 무엇을 하던 남들보다 꼼꼼하게 해내고 싶은 마음에 무엇이든 신중하고 ...
2,때문에 무슨 일이든 시작한다면 도중에 무책임하게 피하지 않고 끝까지 맡은 일을 진행...
3,이러한 성격은 공동체 생활에 있어 타인과의 소통과 이를 공감해주는 포용을 중요하게 ...
4,"위와 같은 관점으로 행동하기 때문에 간혹 자신의 주관이 흐려질 우려가 있지만, 타인..."


valid 데이터 분리 함수

In [ ]:
def set_valid(data):
  df_train = data.sample(frac=0.8, random_state=42)
  df_valid = data.drop(df_train.index).reset_index(drop=True)
  return df_valid

valid_c = set_valid(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/c_final_real.csv'))
valid_e = set_valid(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/e_final_real.csv'))
valid_n = set_valid(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/n_final_real.csv'))

학습 모델 load

In [ ]:
modelC_path = '/content/drive/MyDrive/BertModel/BertModel_C'
modelE_path = '/content/drive/MyDrive/BertModel/BertModel_E'
modelN_path = '/content/drive/MyDrive/BertModel/BertModel_N'

predictorC = TextPredictor.load(modelC_path)
predictorE = TextPredictor.load(modelE_path)
predictorN = TextPredictor.load(modelN_path)

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/BertModel/BertModel_C/model.ckpt
INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/BertModel/BertModel_E/model.ckpt
INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/BertModel/BertModel_N/model.ckpt


모델 성능 평가

In [ ]:
predC = predictorC.evaluate(valid_c, metrics=['accuracy', 'f1'])
predE = predictorE.evaluate(valid_e, metrics=['accuracy', 'f1'])
predN = predictorN.evaluate(valid_n, metrics=['accuracy', 'f1'])
print('CON 분류 성능:', predC)
print('EXT 분류 성능:', predE)
print('NEU 분류 성능:', predN)

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

CON 분류 성능: {'accuracy': 0.5388252332069685, 'f1': 0.6280020615014603}
EXT 분류 성능: {'accuracy': 0.5313271473567719, 'f1': 0.4729309271935283}
NEU 분류 성능: {'accuracy': 0.5412190963341859, 'f1': 0.6036165432917173}


성격 예측 모델: 3가지 예측 모델 병합

In [ ]:
def BertPredictor(data, textCol: str):
  target = data.rename(columns = {textCol: 'KOR_TEXT'})
  target['cCON'] = predictorC.predict(target[['KOR_TEXT']])
  target['cEXT'] = predictorE.predict(target[['KOR_TEXT']])
  target['cNEU'] = predictorN.predict(target[['KOR_TEXT']])
  target.rename(columns = {'KOR_TEXT': textCol}, inplace=True)
  return target

test 데이터에 예측 모델 적용

In [ ]:
data = BertPredictor(test, 'A')
data

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

,A,cCON,cEXT,cNEU
0,꾸준함꾸준함은 저 자신을 한 단어로 정의한다고 할 때 가장 먼저 표현할 수 있는 단...,y,n,y
1,어렸을 때부터 무엇을 하던 남들보다 꼼꼼하게 해내고 싶은 마음에 무엇이든 신중하고 ...,y,n,y
2,때문에 무슨 일이든 시작한다면 도중에 무책임하게 피하지 않고 끝까지 맡은 일을 진행...,y,n,n
3,이러한 성격은 공동체 생활에 있어 타인과의 소통과 이를 공감해주는 포용을 중요하게 ...,y,n,n
4,"위와 같은 관점으로 행동하기 때문에 간혹 자신의 주관이 흐려질 우려가 있지만, 타인...",y,n,n
...,...,...,...,...
11794,"고교 3학년 선생님이 학급임원이 아니었던 저에게 학급공금을 맡기고 관리하시면서, 네...",y,n,n
11795,온정이 미덕인 우리나라 풍조에서 종종 저에게 편법적인 요구를 하시는 분들이 있었습니다 .,y,n,y
11796,저보다 연장자이시거나 상급자일 때는 더욱 어렵습니다.,y,n,n
11797,사소한 것이라 생각하셔서 혹은 문제없을 것이란 이유로 본인 편의를 봐주시길 바라시는...,y,n,y


예측 데이터 save

In [ ]:
data.to_csv('/content/drive/MyDrive/Colab Notebooks/BERT_자소서_분류.csv', encoding='utf-8-sig', index=False)